In [ ]:
from windIO.utils.yml_utils import validate_yaml
from flow_api import run_foxes, run_pywake
import os
import xarray as xr
from windIO.utils import plant_schemas_path
import matplotlib.pyplot as plt

In [ ]:
yml_input = '../examples/cases/windio_4turbines/wind_energy_system/FLOW_toy_study_wind_energy_system.yaml'
os.makedirs('output', exist_ok=True)

In [ ]:
validate_yaml(yml_input, plant_schemas_path + 'wind_energy_system.yaml')

In [ ]:
run_foxes(yml_input)

In [ ]:
foxes_dat = xr.load_dataset('./results/turbine_data.nc')
foxes_dat 

In [ ]:
run_pywake(yml_input)

In [ ]:
pywake_dat = xr.load_dataset('output/PowerTable.nc')
pywake_dat

In [ ]:
plt.plot(pywake_dat.turbine, pywake_dat.power.mean('time'),c='orange', label="pywake")
plt.plot(foxes_dat.turbine, foxes_dat.power.mean('time') / 1e3, ls='-.',c='blue', label="foxes")
plt.xlabel('Turbine Number')
plt.ylabel('Power (kW)')
plt.legend()
plt.show()

In [ ]:
plt.scatter(pywake_dat.wd, pywake_dat.sel(turbine=1).power, c='orange', marker='o',alpha=0.8, label="pywake")
plt.scatter(pywake_dat.wd, foxes_dat.sel(turbine=1).power / 1e3, c='blue', marker='x',alpha=0.5, label="foxes")
plt.xlabel('Wind direction [°]')
plt.ylabel('Power (kW)')
plt.legend()
plt.show()